In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio cmd pip install v3io

    100% |████████████████████████████████| 153kB 42.8MB/s ta 0:00:01
    100% |████████████████████████████████| 921kB 20.0MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 21.7MB/s ta 0:00:01
    100% |████████████████████████████████| 184kB 47.5MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 34.0MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 34.0MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 34.8MB/s ta 0:00:01
kfp 0.2.5 has requirement urllib3<1.25,>=1.15, but you'll have urllib3 1.25.8 which is incompatible.
distributed 2.10.0 has requirement dask>=2.9.0, but you'll have dask 1.1.5 which is incompatible.
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found existing installation: Pygments 2.5.2
    Uninstalling Pygments-2.5.2:
      Successfully uninstalled Pygments-2.5.2
  Found existing installation: idna 2.8
    Uninstalling idna-2.8:
  

In [ ]:
%%nuclio config
# Base image
spec.build.baseImage = "python:3.6-jessie"
spec.triggers.httpworkers.kind = "http"
spec.triggers.httpworkers.maxWorkers  = 32
spec.triggers.httpworkers.workerAvailabilityTimeoutMilliseconds = 1000

In [3]:
%%nuclio env 
V3IO_API=http://v3io-webapi:8081
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
CONTAINER=development
TABLE_PATH=/padsquad/simpli_events_optimized
NUM_THREADS=16

%nuclio: setting 'V3IO_API' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'CONTAINER' environment variable
%nuclio: setting 'TABLE_PATH' environment variable
%nuclio: setting 'NUM_THREADS' environment variable


In [4]:
import os
import queue
import threading
import datetime
import uuid

import v3io.dataplane

In [9]:
def handler(context, event):
    # shove event into queue to be handled by the threads
    context.event_queue.put(event)

    # immediately return with an "ACCEPTED" response
    return context.Response(status_code=202,
                            headers={
                                "Access-Control-Allow-Origin": "*",
                                "Access-Control-Allow-Headers": "cache-control, content-type"
                            })

In [6]:
def init_context(context):
    # create configuration
    setattr(context, 'config', {
        'num_threads': os.environ.get('NUM_THREADS', 16),
        'container': os.environ.get('CONTAINER', 'users'),
        'path': os.environ.get('TABLE_PATH', '/padsquad/test'),
    })

    # create queue and thread pool
    setattr(context, 'event_queue', queue.Queue())
    setattr(context, 'thread_pool', _create_thread_pool(context))


def _create_thread_pool(context):
    thread_pool = []
    for thread_idx in range(int(context.config['num_threads'])):
        thread = threading.Thread(target=_thread_entry, args=(context,))
        thread.start()

        # add to pool
        thread_pool.append(thread)


def _thread_entry(context):
    # create our very own v3io client with 1 connection towards webapi, since we
    # don't plan on doing any parallel requests within the thread
    v3io_client = v3io.dataplane.Client(context.logger, max_connections=1)

    while True:
        event = context.event_queue.get()
        if event is None:
            break

        try:
            _store_event(context, v3io_client, event)
        except Exception as e:
            context.logger.warn_with('Failed to store event', e=e)


def _store_event(context, v3io_client, event):
    now = datetime.datetime.now()

    record_id = str(uuid.uuid4())

    # store in v3io
    v3io_client.put_item(container=context.config['container'],
                         path=os.path.join(context.config['path'], record_id),
                         attributes={
                             'id': record_id,
                             'date_year': now.year,
                             'date_month': now.month,
                             'date_day': now.day,
                             'date_hour': now.hour,
                             'CPProcess_IND': 0,
                             'DOProcess_IND': 0,
                             'who': event.body.get('who', ''),
                             'label': event.body.get('label', ''),
                             'assetName': event.body.get('assetName', ''),
                             'target': event.body.get('target', ''),
                             'direction': event.body.get('direction', ''),
                             'impressionID': event.body.get('impressionID', ''),
                             'simpliCampaignId': event.body.get('simpliCampaignId', ''),
                             'simpliPlacementId': event.body.get('simpliPlacementId', ''),
                             'celtraCreativeId': event.body.get('celtraCreativeId', ''),
                             'type': event.body.get('type', ''),
                         })

In [7]:
%nuclio deploy -p iguazio-optimized -n simpli-events-optimized -c

[nuclio] 2020-03-28 22:45:38,533 (info) Building processor image
[nuclio] 2020-03-28 22:45:43,590 (info) Build complete
[nuclio] 2020-03-28 22:45:47,697 (info) Function deploy complete
[nuclio] 2020-03-28 22:45:47,708 done creating simpli-events-optimized, function address: 3.87.145.179:30981
%nuclio: function deployed


In [ ]:
%nuclio show